In [ ]:
# !pip install simplemma
# !pip install sumy

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from simplemma import text_lemmatizer
from nltk import sent_tokenize
import numpy as np
import string
import re

In [ ]:
stop_words = None
with open("stopwords.txt", "r") as in_file:
    stop_words = set(in_file.read().splitlines())

In [ ]:
doc = None
with open("metin_1.txt", "r") as in_file:
    doc = in_file.read()

baslik = doc.split("\n", 1)[0]
metin = doc.split("\n", 1)[1]

print("Başlık:")
print(baslik)

print("*" * 50)

print("Metin:")
print(metin)

In [ ]:
cumleler = sent_tokenize(metin)
cumleler

In [ ]:
def clean_text(text):
    text = text.lower().translate(str.maketrans("", "", string.punctuation))
    text = re.sub("’|“|”|‘|–|—", " ", text)
    text = text_lemmatizer(text, lang="tr")
    return " ".join([word for word in text if word not in stop_words])

In [ ]:
cumleler_temiz = [clean_text(c) for c in cumleler]
cumleler_temiz

In [ ]:
vectorizer = TfidfVectorizer(norm="l1")

In [ ]:
X = vectorizer.fit_transform(cumleler_temiz)

In [ ]:
v1 = X[0]
v1.toarray()

In [ ]:
v1[v1 != 0]

In [ ]:
v1[v1 != 0].mean()

In [ ]:
def cumle_puani(vektor):
    non_zero_elems = vektor[vektor != 0]
    return non_zero_elems.mean()

In [ ]:
skorlar = np.array([cumle_puani(v) for v in X])
skorlar

In [ ]:
siralama_indeksleri = np.argsort(-skorlar)
siralama_indeksleri

In [ ]:
print("Başlık:")
print(baslik)
print("Özet:")
for i in siralama_indeksleri[:5]:
    print(cumleler[i])

In [ ]:
def metin_ozetle(dosya_ismi, cumle_sayisi=5):
    doc = None
    with open(dosya_ismi, "r") as in_file:
        doc = in_file.read()

    baslik = doc.split("\n", 1)[0]
    metin = doc.split("\n", 1)[1]
    cumleler = sent_tokenize(metin)
    cumleler_temiz = [clean_text(c) for c in cumleler]
    vectorizer = TfidfVectorizer(norm="l1")
    X = vectorizer.fit_transform(cumleler_temiz)
    skorlar = np.array([cumle_puani(v) for v in X])
    siralama_indeksleri = np.argsort(-skorlar)

    print(baslik)
    print("Özet:")
    for i in siralama_indeksleri[:cumle_sayisi]:
        print(cumleler[i])

In [ ]:
metin_ozetle("metin_2.txt")

Hazır kullanım için metin özetleyiciler


In [ ]:
# https://radimrehurek.com/gensim_3.8.3/auto_examples/tutorials/run_summarization.html

from sumy.summarizers.text_rank import TextRankSummarizer
from sumy.summarizers.lsa import LsaSummarizer
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer

In [ ]:
ozetleyici = TextRankSummarizer()
parser = PlaintextParser.from_string(metin, Tokenizer("tr"))
ozet = ozetleyici(parser.document, sentences_count=5)
for cumle in ozet:
    print(str(cumle))

In [ ]:
ozetleyici = LsaSummarizer()
ozet = ozetleyici(parser.document, sentences_count=5)
for cumle in ozet:
    print(str(cumle))

In [ ]:
def sumy_ile_ozetle(dosya, method="text_rank", cumle_sayisi=5):
    doc = None
    with open(dosya, "r") as in_file:
        doc = in_file.read()

    baslik = doc.split("\n", 1)[0]
    metin = doc.split("\n", 1)[1]

    parser = PlaintextParser.from_string(metin, Tokenizer("tr"))

    ozetleyici = None
    if method == "text_rank":
        ozetleyici = TextRankSummarizer()
    if method == "lsa":
        ozetleyici = LsaSummarizer()

    ozet = ozetleyici(parser.document, sentences_count=cumle_sayisi)
    print(baslik, "\n")
    for cumle in ozet:
        print(str(cumle))

In [ ]:
sumy_ile_ozetle("metin_2.txt", method="text_rank")

In [ ]:
sumy_ile_ozetle("metin_2.txt", method="lsa")

In [39]:
# Abstractive Yöntem
# https://github.com/alisafaya/mukayese/tree/main

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [40]:
tokenizer = AutoTokenizer.from_pretrained("mukayese/mt5-base-turkish-sum")
model = AutoModelForSeq2SeqLM.from_pretrained("mukayese/mt5-base-turkish-sum")

In [41]:
article = """Fransız devi PSG'nin üzerindeki kara bulutlar dağılmıyor. 
Devler Ligi'nde Real Madrid'e olaylı şekilde boyun eğen başkent temsilcisinde oyuncuların gruplaşmaya başladığı öne sürüldü. 
Güney Amerikalılar ve Fransızca konuşanlar olarak ikiye ayrılan oyuncuların saha içerisinde de birbirlerine uzak olduğu iddia edildi. 
İşte PSG'de soyunma odasında yaşananlar ve 20 milyon avroluk tazminat ihtimali... 
UEFA Şampiyonlar Ligi'nde Real Madrid'e sansasyonel bir şekilde elenen Paris Saint Germain'de Kylian Mbappe haricindeki tüm oyunculara yönelik taraftar tepkisinin devam etmesi başkent temsilcisindeki krizi derinleştirdi.
RMC Sport'ta yer alan haberde;
Paris Saint Germain'in soyunma odasında işlerin yolunda gitmediği ve futbolcuların iki gruba ayrıldığı öne sürüldü. İddiaya göre oyuncular gruplaşmaya başladı ve aralarındaki iletişim her geçen gün zayıflıyor."""

inputs = tokenizer([article], max_length=1024, return_tensors="pt")
summary_ids = model.generate(inputs["input_ids"], num_beams=6, max_length=100)
tokenizer.batch_decode(summary_ids, skip_special_tokens=True)[0]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


"UEFA Şampiyonlar Ligi'nde Real Madrid'e olaylı şekilde boyun eğen Paris Saint Germain'de oyuncuların gruplaşmaya başladığı öne sürüldü."